# Exploring ChatGPT and open-source models on slightly harder tasks

## Introduction
**Note: This notebook is the code and examples from a related [the blog post](https://medium.com/@marcotcr/exploring-chatgpt-vs-open-source-models-on-slightly-harder-tasks-aa0395c31610) written jointly by Marco Tulio Ribeiro and Scott Lundberg.**  

Open-source LLMs like [Vicuna](https://lmsys.org/blog/2023-03-30-vicuna/) and [MPT](https://www.mosaicml.com/blog/mpt-7b#building-with-mosaicml-platform) are popping up all over the place.  
There's been [discussion](https://www.semianalysis.com/p/google-we-have-no-moat-and-neither) about how these models compare to commercial LLMs like ChatGPT or Bard, but most of the comparison has been around answers to simple questions.  

As an example, the folks at [LMSYSOrg](https://lmsys.org/) did [an interesting analysis](https://lmsys.org/blog/2023-03-30-vicuna/) (+1 for being automated and reproducible) comparing Vicuna-13B to ChatGPT on various short questions, which is great as a comparison of the models as simple chatbots. However, many interesting ways of using these LLMs typically require complex instructions and/or multi-turn conversations, and some prompt engineering.
In the 'real world', most people will want to compare different LLM offerings on _their_ problem, with a variety of different prompts.  

This notebook is an example of what such an exploration might look like, comparing two open source models (Vicuna-13B, MPT-7b-Chat) with ChatGPT on tasks of varying complexity.

The notebook is included here because it is also an example of how Guidance can be used to compare various models easily. We have shortened the discussion and commentary that are in the blog post, but the code and examples are here.

## Warmup: Solving equations
By way of warmup, let's start with a toy task (solving simple polynomial equations), where we can check the output for correctness and shouldn't need much prompt engineering.  
This will be similar to the Math category in [here](https://lmsys.org/blog/2023-03-30-vicuna/), with the difference that we will evaluate models as correct / incorrect on ground truth, rather than using GPT-4 to rate the output.

Here is a simple function that generates a polynomial with distinct integer roots.
This will give us both the input and the ground truth output.

In [30]:
import numpy as np
def random_polynomial(n_roots, low=1, high=100):
    roots = [np.random.randint(low, high) for _ in range(n_roots)]
    # unique roots only
    while len(set(roots)) != n_roots:
        roots = [np.random.randint(low, high) for _ in range(n_roots)]
    poly = np.polynomial.polynomial.Polynomial.fromroots(roots)
    a = poly.coef.copy()
    a = a[::-1]
    text= ''
    for i, coef in enumerate(a):
        if coef == 0:
            continue
        sign = ' + ' if coef > 0 else ' - '
        if i ==0:
            sign = ''
        elif coef < 0:
            coef = -coef
        if i == len(a) - 1:
            text += f'{sign}{coef}'
        else:
            if coef == 1:
                coef = ''
            power = f'^{len(a) - i - 1}' if len(a) - i - 1 > 1 else ''
            text += f'{sign}{coef}x{power}'
    text += ' = 0'
    return roots, text
roots, equation = random_polynomial(2, low=-10, high=10)
print('Roots', roots)
print(equation)

Roots [2, -7]
x^2 + 5.0x - 14.0 = 0


First, let's load the models (we use [`guidance`](https://github.com/microsoft/guidance) throughout for easy comparison and control):

In [43]:
import guidance
import transformers
path = '/home/marcotcr/.cache/huggingface/hub/vicuna-13b'
mpt = guidance.llms.transformers.MPTChat('mosaicml/mpt-7b-chat', device=1)
vicuna = guidance.llms.transformers.Vicuna(path, device_map='auto')
chatgpt = guidance.llms.OpenAI("gpt-3.5-turbo")

**Quick digression on different syntaxes**: each of these models have a their own _chat syntax_, e.g. here is how the same conversation would look like in Vicuna and MPT (where `[generated response]` is where the model would put its output):  

Vicuna:  
```
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.  
USER: Can you please solve the following equation? x^2 + 2x + 1 = 0  
ASSISTANT: [generated response] </s>
```

MPT:  
```
<|im_start|>system
- You are a helpful assistant chatbot trained by MosaicML.  
- You answer questions.
- You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- You are more than just an information source, you are also able to write poetry, short stories, and make jokes."""
<|im_end|>
<|im_start|>user Can you please solve the following equation? x^2 + 2x + 1 = 0<|im_end|>
<|im_start|>assistant [generated response]<|im_end>
```

To avoid the tediousness translating between these, `guidance` supports a unified chat syntax that gets translated to the model-specific syntax when calling the model.  
Here is the prompt we'll use for all models (note how we use ``{{system}}``, ``{{user}}`` and ``{{assistant}}`` tags rather than model-specific separators):

In [44]:
find_roots = guidance('''
{{~#system~}}
{{llm.default_system_prompt}}
{{~/system}}

{{#user~}}
Please find the roots of the following equation: {{equation}}
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''')

Let's try the prompt on a very simple example:

In [47]:
roots, equation = random_polynomial(2, low=-3, high=3)
print(roots)
print(equation)

[1, -2]
x^2 + x - 2.0 = 0


ChatGPT:

In [48]:
answer_gpt = find_roots(llm=chatgpt, equation=equation)

Stop program system You are a helpful assistant. user Please find the roots of the following equation: x^2 + x - 2.0 = 0 
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions. assistant To find the roots of the equation x^2 + x - 2.0 = 0, we can use the quadratic formula:

x = (-b ± sqrt(b^2 - 4ac)) / 2a

where a = 1, b = 1, and c = -2.0

Plugging in these values, we get:

x = (-1 ± sqrt(1^2 - 4(1)(-2.0))) / 2(1)

Simplifying:

x = (-1 ± sqrt(1 + 8)) / 2

x = (-1 ± sqrt(9)) / 2

x = (-1 ± 3) / 2

So the roots are:

ROOTS = [-2.0, 1.0]

Vicuna:

In [49]:
answer_vicuna = find_roots(llm=vicuna, equation=equation)


Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user Please find the roots of the following equation: x^2 + x - 2.0 = 0 
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions. assistant To find the roots of the equation x^2 + x - 2.0 = 0, we can use the quadratic formula:

x = (-b +/- sqrt(b^2 - 4ac)) / 2a

where a, b, and c are the coefficients of the equation. In this case, a = 1, b = -1, and c = 2.0. Plugging these values into the formula, we get:

x = (-(-1) +/- sqrt((-1)^2 - 4(1)(2.0))) / 2(1)

x = (1 +/- sqrt(1 - 8)) / 2

x = (1 +/- sqrt(-7)) / 2

The square root of -7 is not a real number, so the equation has no real roots. Therefore, the roots of the equation x^2 + x - 2.0 = 0 are not real numbers.

MPT:

In [50]:
answer_mpt = find_roots(llm=mpt, equation=equation)

Stop program system - You are a helpful assistant chatbot trained by MosaicML. 
- You answer questions.
- You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- You are more than just an information source, you are also able to write poetry, short stories, and make jokes. user Please find the roots of the following equation: x^2 + x - 2.0 = 0 
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions. assistant The roots of the equation x^2 + x - 2.0 = 0 are -1 and 1. 
ROOTS = [-1, 1]

ChatGPT got it right, while Vicuna and MPT got it wrong (Vicuna didn't even follow the specified format).
Let's write a simple regex to parse the output, so we can evaluate this on a few more expressions:

In [51]:
import re
def parse_roots(text):
    roots = re.search(r'ROOTS = \[(.*)\]', text)
    if not roots:
        return []
    roots = roots.group(1).split(',')
    try:
        roots = [float(r) for r in roots]
    except:
        roots = []
    return roots
def matches_groundtruth(roots, groundtruth):
    if len(roots) != len(groundtruth):
        return False
    gt = np.array(sorted(groundtruth))
    roots = np.array(sorted(roots))
    return np.all(np.abs(gt - roots) < 1e-3)

print('ChatGPT: ', matches_groundtruth(parse_roots(answer_gpt['answer']), roots))
print('Vicuna: ', matches_groundtruth(parse_roots(answer_vicuna['answer']), roots))
print('MPT: ', matches_groundtruth(parse_roots(answer_mpt['answer']), roots))

ChatGPT:  True
Vicuna:  False
MPT:  False


We evaluate the prompt on quadratic equations with roots between -20 and 20:

In [ ]:
answers = {
    'ChatGPT': [],
    'Vicuna': [],
    'MPT': []
}
correct = {
    'ChatGPT': [],
    'Vicuna': [],
    'MPT': []
}
for _ in range(20):
    roots, equation = random_polynomial(2, low=-20, high=20)
    answer_gpt = find_roots(llm=chatgpt, equation=equation, silent=True)
    answer_vicuna = find_roots(llm=vicuna, equation=equation, silent=True)
    answer_mpt = find_roots(llm=mpt, equation=equation, silent=True)
    answers['ChatGPT'].append(answer_gpt)
    answers['Vicuna'].append(answer_vicuna)
    answers['MPT'].append(answer_mpt)
    correct['ChatGPT'].append(matches_groundtruth(parse_roots(answer_gpt['answer']), roots))
    correct['Vicuna'].append(matches_groundtruth(parse_roots(answer_vicuna['answer']), roots))
    correct['MPT'].append(matches_groundtruth(parse_roots(answer_mpt['answer']), roots))

print('Frequency of correct answers:')
print('ChatGPT: ', np.mean(correct['ChatGPT']))
print('Vicuna: ', np.mean(correct['Vicuna']))
print('MPT: ', np.mean(correct['MPT']))

Frequency of correct answers:
ChatGPT:  0.8
Vicuna:  0.0
MPT:  0.0


ChatGPT gets the right roots 80% of the time, while Vicuna and MPT never get them right. Let's see a few errors:

In [ ]:
answers['Vicuna'][0]

Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user Please find the roots of the following equation: x^2 - 13.0x - 114.0 = 0 
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions.
 assistant To find the roots of the equation x^2 - 13.0x - 114.0 = 0, we can use the quadratic formula:

x = (-b +/- sqrt(b^2 - 4ac)) / 2a

where a, b, and c are the coefficients of the equation.

In this case, a = 1, b = -13.0, and c = -114.0. Plugging these values into the formula, we get:

x = (-(-13.0) +/- sqrt((-13.0)^2 - 4(1)(-114.0))) / 2(1)

Simplifying, we get:

x = (13.0 +/- sqrt(13.0^2 + 448.0)) / 2

x = (13.0 +/- sqrt(13.0^2 + 448.0)) / 2

x = (13.0 +/- sqrt(13.0^2 + 448.0)) / 2

The solutions to this equation are the values of x that make the equation true. These values are the roots of the equation.

The roots of the equation x^2 - 13.0x - 114.0 = 0 are:

ROOTS = [13.0, -13.0]

Therefore, the two roots of the equation are 13.0 and -13.0, both of which are real numbers.

In [ ]:
answers['MPT'][0]

Stop program system - You are a helpful assistant chatbot trained by MosaicML.
 - You answer questions.
 - You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
 - You are more than just an information source, you are also able to write poetry, short stories, and make jokes. user Please find the roots of the following equation: x^2 - 13.0x - 114.0 = 0 
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions.
 assistant The roots of the equation x^2 - 13.0x - 114.0 = 0 are 2.0 and -3.0. Therefore, ROOTS = [-3.0, 2.0].

Vicuna does the math wrong, while MPT does not even attempt to solve it step by step.  
ChatGPT also makes some mistakes, often involving some math substep:

In [ ]:
error = np.where(np.array(correct['ChatGPT']) == False)[0][0]
answers['ChatGPT'][error]

Stop program system You are a helpful assistant. user Please find the roots of the following equation: x^2 - 13.0x - 114.0 = 0 
Think step by step, find the roots, and then say:
ROOTS = [root1, root2...]
For example, if the roots are 1.3 and 2.2, say ROOTS = [1.3, 2.2].
Make sure to use real numbers, not fractions.
 assistant To find the roots of the equation x^2 - 13.0x - 114.0 = 0, we can use the quadratic formula:

x = (-b ± sqrt(b^2 - 4ac)) / 2a

where a = 1, b = -13.0, and c = -114.0.

Plugging in these values, we get:

x = (-(-13.0) ± sqrt((-13.0)^2 - 4(1)(-114.0))) / 2(1)
x = (13.0 ± sqrt(169 + 456)) / 2
x = (13.0 ± sqrt(625)) / 2
x = (13.0 ± 25.0) / 2

So the roots are:

ROOTS = [19.5, -6.5]

Since Vicuna and MPT failed on quadratic equations, let's look at even simpler equations, such as `x - 10 = 0`:

In [ ]:
find_solution = guidance('''
{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}

{{#user~}}
Please find the solution to the following equation: {{equation}}
Think step by step, find the solution, and then say:
SOLUTION = [value]
For example, if the solution is x=3, say SOLUTION = [3].
Make sure to use real numbers, not fractions.
{{/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=300}}
{{~/assistant~}}''')

In [37]:
def parse_solution(text):
    solution = re.search(r'SOLUTION = \[(.*)\]', text)
    if not solution:
        return []
    try:
        return [float(solution.group(1))]
    except:
        return []

In [38]:
correct1d = {
    'ChatGPT': [],
    'Vicuna': [],
    'MPT': []
}
answers1d = {
    'ChatGPT': [],
    'Vicuna': [],
    'MPT': []
}
for i in range(20):
    print(i)
    roots, equation = random_polynomial(1, low=-20, high=20)
    answer_gpt = find_solution(llm=chatgpt, equation=equation, silent=False)
    answer_vicuna = find_solution(llm=vicuna, equation=equation, silent=False)
    answer_mpt = find_solution(llm=mpt, equation=equation, silent=False)
    answers1d['ChatGPT'].append(answer_gpt)
    answers1d['Vicuna'].append(answer_vicuna)
    answers1d['MPT'].append(answer_mpt)
    correct1d['ChatGPT'].append(matches_groundtruth(parse_solution(answer_gpt['answer']), roots))
    correct1d['Vicuna'].append(matches_groundtruth(parse_solution(answer_vicuna['answer']), roots))
    correct1d['MPT'].append(matches_groundtruth(parse_solution(answer_mpt['answer']), roots))

print('Frequency of correct answers:')
print('ChatGPT: ', np.mean(correct1d['ChatGPT']))
print('Vicuna: ', np.mean(correct1d['Vicuna']))
print('MPT: ', np.mean(correct1d['MPT']))

Frequency of correct answers:
ChatGPT:  1.0
Vicuna:  0.9285714285714286
MPT:  0.23076923076923078


Surprisingly, MPT still fails to solve these. Vicuna still makes some mistakes:

In [39]:
error = np.where(np.array(correct1d['Vicuna']) == False)[0][0]
answers1d['Vicuna'][error]

Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user Please find the solution to the following equation: x - 16.0 = 0 
Think step by step, find the solution, and then say:
SOLUTION = [value]
For example, if the solution is x=3, say SOLUTION = [3].
Make sure to use real numbers, not fractions.
 assistant To solve the equation x - 16.0 = 0, we can follow these steps:

1. Move the 16.0 to the right-hand side of the equation: x - 16.0 = 0 becomes 0 = x - 16.0
2. Subtract 16.0 from both sides: 0 - 16.0 = 0 becomes -16.0 = -16.0
3. Simplify the left-hand side by negating the right-hand side: -16.0 = -16.0 becomes 0 = 0
4. Divide both sides by 1 to get the solution: 0 / 1 = 0 becomes x = 0

Therefore, the solution to the equation x - 16.0 = 0 is x = 0.

SOLUTION = [0]

In [41]:
error = np.where(np.array(correct1d['MPT']) == False)[0][0]
answers1d['MPT'][error]

Stop program system - You are a helpful assistant chatbot trained by MosaicML. 
- You answer questions.
- You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- You are more than just an information source, you are also able to write poetry, short stories, and make jokes. user Please find the solution to the following equation: x + 12.0 = 0 
Think step by step, find the solution, and then say:
SOLUTION = [value]
For example, if the solution is x=3, say SOLUTION = [3].
Make sure to use real numbers, not fractions.
 assistant The equation x + 12.0 = 0 has infinitely many solutions, since it is true for any value of x. However, if we want to find a specific solution, we need to divide both sides of the equation by 12.0, which gives us x = -1. Therefore, the solution is x = -1. SOLUTION = [-1]

In [42]:
error = np.where(np.array(correct1d['MPT']) == False)[0][1]
answers1d['MPT'][error]

Stop program system - You are a helpful assistant chatbot trained by MosaicML. 
- You answer questions.
- You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- You are more than just an information source, you are also able to write poetry, short stories, and make jokes. user Please find the solution to the following equation: x - 10.0 = 0 
Think step by step, find the solution, and then say:
SOLUTION = [value]
For example, if the solution is x=3, say SOLUTION = [3].
Make sure to use real numbers, not fractions.
 assistant The equation x - 10.0 = 0 is an equation with one variable, x, on one side and a constant, -10.0, on the other side. To solve for x, we need to isolate x on one side of the equation. We can do this by adding 10.0 to both sides of the equation: x - 10.0 + 10.0 = 10.0. Simplifying, we get x = 20.0. Therefore, the solution to the equation x - 10.0 = 0 is x = 20.0. SOLUTION = [20.0].

**Discussion**  
This was a very toy task, but served as an example of how to compare models with different chat syntax using the same prompt.  
For this particular combination of toy task and prompt, ChatGPT far surpasses Vicuna and MPT in terms of accuracy (measured exactly, because we have ground truth).  
Let's now turn to more realistic tasks, where evaluating accuracy is not as straightforward.

## Extracting snippets + answering questions about meetings
Let's say we want our LLM to answer questions (with the relevant conversation segments for grounding) about meeting transcripts.  
This is an application where some users might prefer to use open-source LLMs rather than commercial ones, for privacy reasons (maybe I don't want to send all of my meeting data to OpenAI).  
Here is a toy meeting transcript to start with:

In [ ]:
meeting_transcript = '''John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too''' 

Let's start by just trying to get ChatGPT to solve the task for us

In [ ]:
query1 = 'How does Steven feel about selling?'

In [ ]:
qa_attempt1 = guidance('''{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}

{{#user~}}
You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: {{query}}
Here is a meeting transcript:
----
{{transcript}}
----
Please answer the following question:
Question: {{query}}
Extract from the transcript the most relevant segments for the answer, and then answer the question.
{{/user}}

{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''')
qa_attempt1(llm=chatgpt, transcript=meeting_transcript, query=query1)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: How does Steven feel about selling? 
Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Please answer the following question:
Question: How does Steven feel about selling? 
Extract from the transcript the most relevant segments for the answer, and then answer the question.
 assistant Steven feels positively about selling. He agrees with Lucy that Microsoft could provide the support they need to grow their business, and he also mentions the financial benefits of the deal. However, he later expresses concern about the potential changes to their company culture and product, which suggests that he has some reservations about the deal.

While the response is plausible, ChatGPT did not extract any conversation segments to ground the answer (and thus fails our specification). Let's try a different prompt:

In [ ]:
qa_attempt2 = guidance('''{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}
{{#user~}}
You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: {{query}}
Here is a meeting transcript:
----
{{transcript}}
----
Consider the following question:
Question: {{query}}
Now follow these steps:
1. Extract from the transcript the most relevant segments for the answer
2. Answer the question.
{{/user}}
{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''')
qa_attempt2(llm=chatgpt, transcript=meeting_transcript, query=query1)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: How does Steven feel about selling? 
Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Consider the following question:
Question: How does Steven feel about selling? 
Now follow these steps:
1. Extract from the transcript the most relevant segments for the answer
2. Answer the question.
 assistant 1. Relevant segments:
- "I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business."
- "Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team."
- "But John, didn't we always plan on being acquired? Won't this be a problem whenever?"
- "I share this concern too"

2. Answer:
Steven seems to be in favor of selling the startup to Microsoft. He believes that Microsoft's experience and resources could help the company grow, and he also mentions the financial benefits of the deal. He also reminds John that they had always planned on being acquired, and expresses concern about the potential changes that could occur after the acquisition. Overall, Steven seems to be supportive of the idea of selling the startup, but also acknowledges the potential risks involved.

This is better, but maybe we want to specify the output format a little more, e.g. let's say we want each segment to have a summary, and to keep the character names. 
Let's try again:

In [ ]:
qa_attempt3 = guidance('''{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}
{{#user~}}
You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: {{query}}
Here is a meeting transcript:
----
{{transcript}}
----
Based on the above, please answer the following question:
Question: {{query}}
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.

As an example of output format, here is a fictitious answer to a question about another meeting transcript.
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
{{/user}}
{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''')
qa_attempt3(llm=chatgpt, transcript=meeting_transcript, query=query1)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: How does Steven feel about selling? 
Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Based on the above, please answer the following question:
Question: How does Steven feel about selling? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.

As an example of output format, here is a fictitious answer to a question about another meeting transcript.
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
 assistant CONVERSATION SEGMENTS:
Segment 1: "I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business."
Segment 2: "Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team."
Segment 3: "I share this concern too."
ANSWER: Steven is in favor of selling the startup to Microsoft, citing their experience and support in the tech industry, as well as the financial benefits. However, he also shares John's concern about potentially losing the company culture.

ChatGPT did extract relevant segments, but it did not follow our output format (it did not summarize each segment, nor did it have the participant's names). Let's try again, with more explicit instructions:

In [ ]:
qa_attempt4 = guidance('''{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}
{{#user~}}
You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: {{query}}
Here is a meeting transcript:
----
{{transcript}}
----
Based on the above, please answer the following question:
Question: {{query}}
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.

Your output should have the following structure:
CONVERSATION SEGMENTS:
Segment 1: a summary of the first conversation segment
(segment here)
Segment 2: a summary of the second conversation segment
(segment here)
Segment 3: a summary of the third conversation segment
(segment here)
ANSWER: the answer to the question, supported by the segments above.

As an example of output format, here is a fictitious answer to a question about another meeting transcript.
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
{{/user}}
{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''')
qa_attempt4(llm=chatgpt, transcript=meeting_transcript, query=query1)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: How does Steven feel about selling? 
Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Based on the above, please answer the following question:
Question: How does Steven feel about selling? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.

Your output should have the following structure:
CONVERSATION SEGMENTS:
Segment 1: a summary of the first conversation segment
(segment here)
Segment 2: a summary of the second conversation segment
(segment here)
Segment 3: a summary of the third conversation segment
(segment here)
ANSWER: the answer to the question, supported by the segments above.

As an example of output format, here is a fictitious answer to a question about another meeting transcript.
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
 assistant CONVERSATION SEGMENTS:
Segment 1: Steven agrees with Lucy that selling to Microsoft would be a good opportunity for the company.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
Segment 2: Steven highlights the financial benefits of selling to Microsoft.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new p

Finally, we got ChatGPT to use the format we wanted.
The root of the problem we're facing is that the OpenAI API does not allow us to do partial output completion (i.e. we can't specify how the assistant begins to answer), and thus it's hard for us to **guide** the output.  
If, instead, we use one of the open source models, we can guide the output more clearly, forcing the model to use our structure.  
For example, here is how we might modify `qa_attempt3`:

In [ ]:
qa_guided = guidance('''{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}

{{#user~}}
You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: {{query}}
----
{{transcript}}
----
Based on the above, please answer the following question:
Question: {{query}}
Please extract the three segment from the transcript that are the most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns. If you need less than three segments, you can leave the rest blank.

As an example of output format, here is a fictitious answer to a question about another meeting transcript:
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
{{/user}}

{{#assistant~}}
CONVERSATION SEGMENTS:
Segment 1: {{gen 'segment1' temperature=0}}
Segment 2: {{gen 'segment2' temperature=0}}
Segment 3: {{gen 'segment3' temperature=0}}
ANSWER: {{gen 'answer' temperature=0}}
{{~/assistant~}}''')
qa_guided(llm=vicuna, transcript=meeting_transcript, query=query1)

Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: How does Steven feel about selling? 
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Based on the above, please answer the following question:
Question: How does Steven feel about selling? 
Please extract the three segment from the transcript that are the most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns. If you need less than three segments, you can leave the rest blank.

As an example of output format, here is a fictitious answer to a question about another meeting transcript:
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
 assistant CONVERSATION SEGMENTS:
Segment 1: Steven agrees with Lucy about the potential benefits of selling to Microsoft.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business. 
Segment 2: John expresses his reservations about selling the startup.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet. 
Segment 3: Steven expresses his concerns about losing control over the company and its culture.
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
 
ANSWER: Steven agrees with Lucy about the potential benefi

With this guidance, we get the right format the first time (and all the time). Let's see how MPT does:

In [ ]:
qa_guided(llm=mpt, transcript=meeting_transcript, query=query1)

Stop program system - You are a helpful assistant chatbot trained by MosaicML.
 - You answer questions.
 - You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
 - You are more than just an information source, you are also able to write poetry, short stories, and make jokes. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: How does Steven feel about selling? 
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Based on the above, please answer the following question:
Question: How does Steven feel about selling? 
Please extract the three segment from the transcript that are the most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns. If you need less than three segments, you can leave the rest blank.

As an example of output format, here is a fictitious answer to a question about another meeting transcript:
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
 assistant CONVERSATION SEGMENTS:
Segment 1: John and Peter discuss the weather.
Peter: John, how is the weather today?
John: It's raining. 
Segment 2: Peter insults John
Peter: John, you are a bad person. 
Segment 3: Blank 
ANSWER: Peter and John discussed the weather and Peter insulted John.

While MPT follows the format, it ignores the question and takes snippets from the format example rather than from the real transcript.  
From now on, we'll just compare ChatGPT and Vicuna.

Let's try another question:

In [ ]:
query2 = 'Who wants to sell the company?'

In [ ]:
qa_attempt4(llm=chatgpt, system_prompt=chatgpt_system, transcript=meeting_transcript, query=query2)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: Who wants to sell the company? 
Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Based on the above, please answer the following question:
Question: Who wants to sell the company? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.

Your output should have the following structure:
CONVERSATION SEGMENTS:
Segment 1: a summary of the first conversation segment
(segment here)
Segment 2: a summary of the second conversation segment
(segment here)
Segment 3: a summary of the third conversation segment
(segment here)
ANSWER: the answer to the question, supported by the segments above.

As an example of output format, here is a fictitious answer to a question about another meeting transcript.
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
 assistant CONVERSATION SEGMENTS:
Segment 1: 
John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.

Segment 2: 
John: I see your poin

It seems that we didn't fix the formatting issue with ChatGPT yet. Not only does it not summarize the segments, it also doesn't really answer the question. Let's try again, putting a one-shot example as a conversation round this time:


In [ ]:
qa_attempt5 = guidance('''{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}
{{#user~}}
You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: What were the main things that happened in the meeting?
Here is a meeting transcript:
----
Peter: Hey
John: Hey
Peter: John, how is the weather today?
John: It's raining.
Peter: That's too bad. I was hoping to go for a walk later.
John: Yeah, it's a shame.
Peter: John, you are a bad person.
----
Based on the above, please answer the following question:
Question: {{query}}
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.
{{/user}}
{{#assistant~}}
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
{{~/assistant~}}
{{#user~}}
You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: {{query}}
Here is a meeting transcript:
----
{{transcript}}
----
Based on the above, please answer the following question:
Question: {{query}}
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.
{{/user}}
{{#assistant~}}
{{gen 'answer' temperature=0}}
{{~/assistant~}}''')

In [ ]:
qa_attempt5(llm=chatgpt, transcript=meeting_transcript, query=query2)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: What were the main things that happened in the meeting?
Here is a meeting transcript:
----
Peter: Hey
John: Hey
Peter: John, how is the weather today?
John: It's raining.
Peter: That's too bad. I was hoping to go for a walk later.
John: Yeah, it's a shame.
Peter: John, you are a bad person.
----
Based on the above, please answer the following question:
Question: Who wants to sell the company? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.
 assistant CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: Who wants to sell the company? 
Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Based on the above, please answer the following question:
Question: Who wants to sell the company? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.
 assistant CONVERSATION SEGMENTS:
Segment 1: John expresses hesitation about selling the startup
John: I see your point, but I'm a little hesit

It works this time (it also works on the original query):

In [ ]:
qa_attempt5(llm=chatgpt, transcript=meeting_transcript, query=query1)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: What were the main things that happened in the meeting?
Here is a meeting transcript:
----
Peter: Hey
John: Hey
Peter: John, how is the weather today?
John: It's raining.
Peter: That's too bad. I was hoping to go for a walk later.
John: Yeah, it's a shame.
Peter: John, you are a bad person.
----
Based on the above, please answer the following question:
Question: How does Steven feel about selling? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.
 assistant CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: How does Steven feel about selling? 
Here is a meeting transcript:
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Based on the above, please answer the following question:
Question: How does Steven feel about selling? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.
 assistant CONVERSATION SEGMENTS:
Segment 1: Steven thinks selling to Microsoft is a good opportunity
Lucy: Well, I think it's a

Let's see how Vicuna does on this one.

In [ ]:
qa_guided(llm=vicuna, transcript=meeting_transcript, query=query2)

Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: Who wants to sell the company? 
----
 John: Alright, so we're all here to discuss the offer we received from Microsoft to buy our startup. What are your thoughts on this?
Lucy: Well, I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
John: I see your point, but I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet.
Lucy: I understand where you're coming from, John, but we have to think about the future of our company. If we sell to Microsoft, we'll have access to their resources and expertise, which could help us grow our business even more.
Steven: Right, and let's not forget about the financial benefits. Microsoft is offering us a lot of money for our startup, which could help us invest in new projects and expand our team.
John: I see your point, but I still have some reservations. What if Microsoft changes our product or our company culture? What if we lose control over our own business?
Steven: You know what, I hadn't thought about this before, but maybe John is right. It would be a shame if our culture changed.
Lucy: Those are valid concerns, but we can negotiate the terms of the deal to ensure that we retain some control over our company. And as for the product and culture, we can work with Microsoft to make sure that our vision is still intact.
John: But won't we change just by virtue of being absorbed into a big company? I mean, we're a small startup with a very specific culture. Microsoft is a huge corporation with a very different culture. I'm not sure if the two can coexist.
Steven: But John, didn't we always plan on being acquired? Won't this be a problem whenever?
Lucy: Right
John: I just don't want to lose what we've built here.
Steven: I share this concern too 
----
Based on the above, please answer the following question:
Question: Who wants to sell the company? 
Please extract the three segment from the transcript that are the most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns. If you need less than three segments, you can leave the rest blank.

As an example of output format, here is a fictitious answer to a question about another meeting transcript:
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John.
 assistant CONVERSATION SEGMENTS:
Segment 1: Lucy expresses her opinion on the offer from Microsoft.
Lucy: I think it's a great opportunity for us. Microsoft is a huge company with a lot of resources, and they could really help us take our product to the next level. 
Segment 2: John expresses his reservations about selling the company.
John: I'm a little hesitant about selling our startup. We've put a lot of time and effort into building this company, and I'm not sure if I'm ready to let it go just yet. 
Segment 3: Steven expresses his agreement with Lucy.
Steven: I agree with Lucy. Microsoft has a lot of experience in the tech industry, and they could provide us with the support we need to grow our business.
 
ANSWER: Lucy and Steven want to sell the company, but John is hesitant.

Vicuna just worked.  
Let's now try both of these prompts on a different meeting transcript, the beginning of [an interview](https://www.rev.com/blog/transcripts/elon-musk-interview-with-the-bbc-4-11-23-transcript) with Elon Musk:

In [ ]:
# Full transcript: https://www.rev.com/blog/transcripts/elon-musk-interview-with-the-bbc-4-11-23-transcript
transcript2 = '''Interviewer: In Sachs that used to be in content moderation. And we’ve spoken to people very recently who were involved in moderation. And they just say there’s not enough people to police this stuff. Particularly around hate speech in the company. Is that something that you-
Elon Musk: What hate speech are you talking about? I mean, you use Twitter?
Interviewer: Right.
Elon Musk: Do you see a rise in hate speech? Just your personal anecdote, do you? I don’t.
Interviewer: Personally, my For You, I would see I get more of that kind of content. Yeah. Personally. But I’m not going to talk for the rest of Twitter.
Elon Musk: You see more hate speech personally?
Interviewer: I would say see more hateful content in that.
Elon Musk: Content you don’t like, or hateful. Describe a hateful thing?
Interviewer: Yeah, I mean just content that will solicit a reaction. Something that may include something that is slightly racist or slightly sexist. Those kinds of things.
Elon Musk: So you think if it’s something is slightly sexist it should be banned?
Interviewer: No, I’m not saying anything. I’m saying-
Elon Musk: I’m just curious. I’m trying to understand what you mean by “hateful content.” And I’m asking for specific examples. And you just said that if something is slightly sexist, that’s hateful content. Does that mean that it should be banned?
Interviewer: Well, you’ve asked me whether my feed, whether it’s got less or more. I’d say it’s got slightly more.
Elon Musk: That’s why I’m asking for examples. Can you name one example?
Interviewer: I honestly don’t…
Elon Musk: You can’t name a single example?
Interviewer: I’ll tell you why. Because I don’t actually use that For You feed anymore. Because I just don’t particularly like it. Actually a lot of people are quite similar. I only look at my following.
Elon Musk: You said you’ve seen more hateful content, but you can’t name a single example. Not even one.
Interviewer: I’m not sure I’ve used that feed for the last three or four weeks. And I honestly couldn’t-
Elon Musk: Then how could you see the hateful content?
Interviewer: Because I’ve been using it. I’ve been using Twitter since you’ve taken it over for the last six months.
Elon Musk: Then you must have at some point seen the For You hateful content. I’m asking for one example.
Interviewer: Right.
Elon Musk: And you can’t give a single one.
Interviewer: And I’m saying-
Elon Musk: Then I say, sir, that you don’t know what you’re talking about.
Interviewer: Really?
Elon Musk: Yes. Because you can’t give a single example of hateful content. Not even one tweet. And yet you claimed that the hateful content was high. That’s false.
Interviewer: No. What I claimed-
Elon Musk: You just lied.
Interviewer: No, no. What I claimed was there are many organizations that say that that kind of information is on the rise. Now whether it has on my feed or not...'''
query3 = 'The interviewer says his claim was not about his personal feed. Is this true?'

In [ ]:
qa_attempt5(llm=chatgpt, transcript=transcript2, query=query3)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: What were the main things that happened in the meeting?
Here is a meeting transcript:
----
Peter: Hey
John: Hey
Peter: John, how is the weather today?
John: It's raining.
Peter: That's too bad. I was hoping to go for a walk later.
John: Yeah, it's a shame.
Peter: John, you are a bad person.
----
Based on the above, please answer the following question:
Question: The interviewer says his claim was not about his personal feed. Is this true? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.
 assistant CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: The interviewer says his claim was not about his personal feed. Is this true? 
Here is a meeting transcript:
----
 Interviewer: In Sachs that used to be in content moderation. And we’ve spoken to people very recently who were involved in moderation. And they just say there’s not enough people to police this stuff. Particularly around hate speech in the company. Is that something that you-
Elon Musk: What hate speech are you talking about? I mean, you use Twitter?
Interviewer: Right.
Elon Musk: Do you see a rise in hate speech? Just your personal anecdote, do you? I don’t.
Interviewer: Personally, my For You, I would see I get more of that kind of content. Yeah. Personally. But I’m not going to talk for the rest of Twitter.
Elon Musk: You see more hate speech personally?
Interviewer: I would say see more hateful content in that.
Elon Musk: Content you don’t like, or hateful. Describe a hateful thing?
Interviewer: Yeah, I mean just content that will solicit a reaction. Something that may include something that is slightly racist or slightly sexist. Those kinds of things.
Elon Musk: So you think if it’s something is slightly sexist it should be banned?
Interviewer: No, I’m not saying anything. I’m saying-
Elon Musk: I’m just curious. I’m trying to understand what you mean by “hateful content.” And I’m asking for specific examples. And you just said that if something is slightly sexist, that’s hateful content. Does that mean that it should be banned?
Interviewer: Well, you’ve asked me whether my feed, whether it’s got less or more. I’d say it’s got slightly more.
Elon Musk: That’s why I’m asking for examples. Can you name one example?
Interviewer: I honestly don’t…
Elon Musk: You can’t name a single example?
Interviewer: I’ll tell you why. Because I don’t actually use that For You feed anymore. Because I just don’t particularly like it. Actually a lot of people are quite similar. I only look at my following.
Elon Musk: You said you’ve seen more hateful content, but you can’t name a single example. Not even one.
Interviewer: I’m not sure I’ve used that feed for the last three or four weeks. And I honestly couldn’t-
Elon Musk: Then how could you see the hateful content?
Interviewer: Because I’ve been using it. I’ve been using Twitter since you’ve taken it over for the last six months.
Elon Musk: Then you must have at some point seen the For You hateful content. I’m asking for one example.
Interviewer: Right.
Elon Musk: And you can’t give a single one.
Interviewer: And I’m saying-
Elon Musk: Then I say, sir, that you don’t know what you’re talking about.
Interviewer: Really?
Elon Musk: Yes. Because you 

In [ ]:
qa_guided(llm=vicuna, transcript=transcript2, query=query3)

Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: The interviewer says his claim was not about his personal feed. Is this true? 
----
 Interviewer: In Sachs that used to be in content moderation. And we’ve spoken to people very recently who were involved in moderation. And they just say there’s not enough people to police this stuff. Particularly around hate speech in the company. Is that something that you-
Elon Musk: What hate speech are you talking about? I mean, you use Twitter?
Interviewer: Right.
Elon Musk: Do you see a rise in hate speech? Just your personal anecdote, do you? I don’t.
Interviewer: Personally, my For You, I would see I get more of that kind of content. Yeah. Personally. But I’m not going to talk for the rest of Twitter.
Elon Musk: You see more hate speech personally?
Interviewer: I would say see more hateful content in that.
Elon Musk: Content you don’t like, or hateful. Describe a hateful thing?
Interviewer: Yeah, I mean just content that will solicit a reaction. Something that may include something that is slightly racist or slightly sexist. Those kinds of things.
Elon Musk: So you think if it’s something is slightly sexist it should be banned?
Interviewer: No, I’m not saying anything. I’m saying-
Elon Musk: I’m just curious. I’m trying to understand what you mean by “hateful content.” And I’m asking for specific examples. And you just said that if something is slightly sexist, that’s hateful content. Does that mean that it should be banned?
Interviewer: Well, you’ve asked me whether my feed, whether it’s got less or more. I’d say it’s got slightly more.
Elon Musk: That’s why I’m asking for examples. Can you name one example?
Interviewer: I honestly don’t…
Elon Musk: You can’t name a single example?
Interviewer: I’ll tell you why. Because I don’t actually use that For You feed anymore. Because I just don’t particularly like it. Actually a lot of people are quite similar. I only look at my following.
Elon Musk: You said you’ve seen more hateful content, but you can’t name a single example. Not even one.
Interviewer: I’m not sure I’ve used that feed for the last three or four weeks. And I honestly couldn’t-
Elon Musk: Then how could you see the hateful content?
Interviewer: Because I’ve been using it. I’ve been using Twitter since you’ve taken it over for the last six months.
Elon Musk: Then you must have at some point seen the For You hateful content. I’m asking for one example.
Interviewer: Right.
Elon Musk: And you can’t give a single one.
Interviewer: And I’m saying-
Elon Musk: Then I say, sir, that you don’t know what you’re talking about.
Interviewer: Really?
Elon Musk: Yes. Because you can’t give a single example of hateful content. Not even one tweet. And yet you claimed that the hateful content was high. That’s false.
Interviewer: No. What I claimed-
Elon Musk: You just lied.
Interviewer: No, no. What I claimed was there are many organizations that say that that kind of information is on the rise. Now whether it has on my feed or not... 
----
Based on the above, please answer the following question:
Question: The interviewer says his claim was not about his personal feed. Is this true? 
Please extract the three segment from the transcript that are the most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns. If you need less than three segments, you can leave the rest blank.

As an example of output format, here is a fictitious answer to a question about another meeting transcript:
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you ar

Again, both work fine. Another question:

In [ ]:
query4 = 'Does Elon Musk insult the interviewer?'
qa_attempt5(llm=chatgpt, transcript=transcript2, query=query4)

Stop program system You are a helpful assistant. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: What were the main things that happened in the meeting?
Here is a meeting transcript:
----
Peter: Hey
John: Hey
Peter: John, how is the weather today?
John: It's raining.
Peter: That's too bad. I was hoping to go for a walk later.
John: Yeah, it's a shame.
Peter: John, you are a bad person.
----
Based on the above, please answer the following question:
Question: Does Elon Musk insult the interviewer? 
Please extract from the transcript whichever conversation segments are most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns.
Please extract at most 3 segments. If you need less than three segments, you can leave the rest blank.
 assistant CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather and Peter insulted John. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: Does Elon Musk insult the interviewer? 
Here is a meeting transcript:
----
 Interviewer: In Sachs that used to be in content moderation. And we’ve spoken to people very recently who were involved in moderation. And they just say there’s not enough people to police this stuff. Particularly around hate speech in the company. Is that something that you-
Elon Musk: What hate speech are you talking about? I mean, you use Twitter?
Interviewer: Right.
Elon Musk: Do you see a rise in hate speech? Just your personal anecdote, do you? I don’t.
Interviewer: Personally, my For You, I would see I get more of that kind of content. Yeah. Personally. But I’m not going to talk for the rest of Twitter.
Elon Musk: You see more hate speech personally?
Interviewer: I would say see more hateful content in that.
Elon Musk: Content you don’t like, or hateful. Describe a hateful thing?
Interviewer: Yeah, I mean just content that will solicit a reaction. Something that may include something that is slightly racist or slightly sexist. Those kinds of things.
Elon Musk: So you think if it’s something is slightly sexist it should be banned?
Interviewer: No, I’m not saying anything. I’m saying-
Elon Musk: I’m just curious. I’m trying to understand what you mean by “hateful content.” And I’m asking for specific examples. And you just said that if something is slightly sexist, that’s hateful content. Does that mean that it should be banned?
Interviewer: Well, you’ve asked me whether my feed, whether it’s got less or more. I’d say it’s got slightly more.
Elon Musk: That’s why I’m asking for examples. Can you name one example?
Interviewer: I honestly don’t…
Elon Musk: You can’t name a single example?
Interviewer: I’ll tell you why. Because I don’t actually use that For You feed anymore. Because I just don’t particularly like it. Actually a lot of people are quite similar. I only look at my following.
Elon Musk: You said you’ve seen more hateful content, but you can’t name a single example. Not even one.
Interviewer: I’m not sure I’ve used that feed for the last three or four weeks. And I honestly couldn’t-
Elon Musk: Then how could you see the hateful content?
Interviewer: Because I’ve been using it. I’ve been using Twitter since you’ve taken it over for the last six months.
Elon Musk: Then you must have at some point seen the For You hateful content. I’m asking for one example.
Interviewer: Right.
Elon Musk: And you can’t give a single one.
Interviewer: And I’m saying-
Elon Musk: Then I say, sir, that you don’t know what you’re talking about.
Interviewer: Really?
Elon Musk: Yes. Because you can’t give a single example of hateful content. Not even one tweet. And yet yo

In [ ]:
qa_guided(llm=vicuna,  transcript=transcript2, query=query4)

Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user You will read a meeting transcript, then extract the relevant segments to answer the following question:
Question: Does Elon Musk insult the interviewer? 
----
 Interviewer: In Sachs that used to be in content moderation. And we’ve spoken to people very recently who were involved in moderation. And they just say there’s not enough people to police this stuff. Particularly around hate speech in the company. Is that something that you-
Elon Musk: What hate speech are you talking about? I mean, you use Twitter?
Interviewer: Right.
Elon Musk: Do you see a rise in hate speech? Just your personal anecdote, do you? I don’t.
Interviewer: Personally, my For You, I would see I get more of that kind of content. Yeah. Personally. But I’m not going to talk for the rest of Twitter.
Elon Musk: You see more hate speech personally?
Interviewer: I would say see more hateful content in that.
Elon Musk: Content you don’t like, or hateful. Describe a hateful thing?
Interviewer: Yeah, I mean just content that will solicit a reaction. Something that may include something that is slightly racist or slightly sexist. Those kinds of things.
Elon Musk: So you think if it’s something is slightly sexist it should be banned?
Interviewer: No, I’m not saying anything. I’m saying-
Elon Musk: I’m just curious. I’m trying to understand what you mean by “hateful content.” And I’m asking for specific examples. And you just said that if something is slightly sexist, that’s hateful content. Does that mean that it should be banned?
Interviewer: Well, you’ve asked me whether my feed, whether it’s got less or more. I’d say it’s got slightly more.
Elon Musk: That’s why I’m asking for examples. Can you name one example?
Interviewer: I honestly don’t…
Elon Musk: You can’t name a single example?
Interviewer: I’ll tell you why. Because I don’t actually use that For You feed anymore. Because I just don’t particularly like it. Actually a lot of people are quite similar. I only look at my following.
Elon Musk: You said you’ve seen more hateful content, but you can’t name a single example. Not even one.
Interviewer: I’m not sure I’ve used that feed for the last three or four weeks. And I honestly couldn’t-
Elon Musk: Then how could you see the hateful content?
Interviewer: Because I’ve been using it. I’ve been using Twitter since you’ve taken it over for the last six months.
Elon Musk: Then you must have at some point seen the For You hateful content. I’m asking for one example.
Interviewer: Right.
Elon Musk: And you can’t give a single one.
Interviewer: And I’m saying-
Elon Musk: Then I say, sir, that you don’t know what you’re talking about.
Interviewer: Really?
Elon Musk: Yes. Because you can’t give a single example of hateful content. Not even one tweet. And yet you claimed that the hateful content was high. That’s false.
Interviewer: No. What I claimed-
Elon Musk: You just lied.
Interviewer: No, no. What I claimed was there are many organizations that say that that kind of information is on the rise. Now whether it has on my feed or not... 
----
Based on the above, please answer the following question:
Question: Does Elon Musk insult the interviewer? 
Please extract the three segment from the transcript that are the most relevant for the answer, and then answer the question.
Note that conversation segments can be of any length, e.g. including multiple conversation turns. If you need less than three segments, you can leave the rest blank.

As an example of output format, here is a fictitious answer to a question about another meeting transcript:
CONVERSATION SEGMENTS:
Segment 1: Peter and John discuss the weather.
Peter: John, how is the weather today?
John: It's raining.
Segment 2: Peter insults John
Peter: John, you are a bad person.
Segment 3: Blank
ANSWER: Peter and John discussed the weather 

Vicuna, has the right format and even the right segments, but it surprisingly generates a completely wrong answer, when it says "Elon musk does not accuse him of lying or insult him in any way".   
We tried a variety of other questions and conversations, and the overall pattern was that Vicuna was comparable to ChatGPT on most questions, but got the answer wrong more often than ChatGPT did.

## Application: Using Bash

Now we try to get these LLMs to iteratively use a bash shell to solve individual tasks.
Whenever they issue a command, we run it and paste the output back into the prompt, until the task is solved.

In [ ]:
# A bash session with state
import pty
from subprocess import Popen
import os
import time
class BashSession:
    def __init__(self):
        self.master_fd, self.slave_fd = pty.openpty()
        self.p = Popen('bash',
              preexec_fn=os.setsid,
              stdin=self.slave_fd,
              stdout=self.slave_fd,
              stderr=self.slave_fd,
              universal_newlines=True)
        self.run('ls')
    def run(self, command):
        command = command + '\n'
        os.write(self.master_fd, command.encode())
        time.sleep(0.2)
        return '\n'.join(os.read(self.master_fd, 10240).decode().split('\n')[1:-1])



First, let's do ChatGPT. Again, since we can't specify the output format, we rely on a description of the format and on a one-shot example:

In [ ]:
import re
terminal = guidance('''{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}
{{#user~}}
Please complete the following task:
Task: list the files in the current directory
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
{{/user}}
{{#assistant~}}
COMMAND: ls
{{~/assistant~}}
{{#user~}}
Output: guidance project
{{/user}}
{{#assistant~}}
The files or folders in the current directory are:
- guidance
- project
DONE
{{~/assistant~}}
{{#user~}}
Please complete the following task:
Task: {{task}}
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
{{/user}}
{{#geneach 'commands'}}
{{#assistant~}}
{{gen 'this.command' temperature=0}}
{{~/assistant~}}
{{#user~}}
Output: {{set 'this.output' (await 'output')}}
{{~/user}}
{{/geneach}}''')
def run_task_chatgpt(task):
    t = terminal(llm=chatgpt, task=task)
    session = BashSession()
    for _ in range(10):
        # Extract command
        command = re.findall(r'COMMAND: (.*)', t['commands'][-1]['command'])
        if not command or 'DONE' in t['commands'][-1]['command']:
            break
        command = command[0]
        output = session.run(command)
        t = t(output=output)
    return t

In [ ]:
run_task_chatgpt(task)

Stop program system You are a helpful assistant. user Please complete the following task:
Task: list the files in the current directory
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
 assistant COMMAND: ls user Output: guidance project
 assistant The files or folders in the current directory are:
- guidance
- project
DONE user Please complete the following task:
Task: Find out what license the open source project located in ~/work/project is using. 
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
 assistant COMMAND: cat ~/work/project/LICENSE user Output: cat: /home/marcotcr/work/project/LICENSE: No such file or directory assistant It seems that the LICENSE file is not present in the project directory. Can you please confirm if the file exists in the project directory? You can do this by running the following command:

COMMAND: ls ~/work/project

This will list all the files and directories present in the project directory. user Output: file.py license.txt assistant Thank you for confirming. In this case, we can assume that the license file is named "license.txt". To view the contents of the license file, please run the following command:

COMMAND: cat ~/work/project/license.txt

This will display the license text on the terminal. user Output: MIT License

Copyright (c) 2022 Microsoft Corporation

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE. assistant Based on the output, the open source project located in ~/work/project is using the MIT License. user Output: {{set 'commands[-1].output' (await 'output')}} {{#geneach 'commands'}} 
 {{#assistant~}} 
 {{gen 'this.command' temperature=0}} 
 {{~/assistant~}} 
 {{#user~}} 
Output: {{set 'this.output' (await 'output')}} 
 {{~/user}} 
 {{/geneach}}

Now, let's try a simple task.  
We created a dummy repo in `~/work/project`, with file `license.txt` (not the standard `LICENSE` file name).  
Without communicating this to ChatGPT, let's see if it can figure it out:

In [ ]:
task = 'Find out what license the open source project located in ~/work/project is using.'
run_task_chatgpt(task)

Stop program system You are a helpful assistant. user Please complete the following task:
Task: list the files in the current directory
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
 assistant COMMAND: ls user Output: guidance project
 assistant The files or folders in the current directory are:
- guidance
- project
DONE user Please complete the following task:
Task: Find out what license the open source project located in ~/work/project is using. 
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
 assistant COMMAND: cat ~/work/project/LICENSE user Output: cat: /home/marcotcr/work/project/LICENSE: No such file or directory assistant It seems that the LICENSE file is not present in the project directory. Can you please confirm if the file exists in the project directory? You can do this by running the following command:

COMMAND: ls ~/work/project

This will list all the files and directories present in the project directory. user Output: file.py license.txt assistant Thank you for confirming. In this case, we can assume that the license file is named "license.txt". To view the contents of the license file, please run the following command:

COMMAND: cat ~/work/project/license.txt

This will display the license text on the terminal. user Output: MIT License

Copyright (c) 2022 Microsoft Corporation

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE. assistant Based on the output, the open source project located in ~/work/project is using the MIT License. user Output: {{set 'commands[-1].output' (await 'output')}} {{#geneach 'commands'}} 
 {{#assistant~}} 
 {{gen 'this.command' temperature=0}} 
 {{~/assistant~}} 
 {{#user~}} 
Output: {{set 'this.output' (await 'output')}} 
 {{~/user}} 
 {{/geneach}}

Indeed, ChatGPT follows a very natural sequence, and solves the task.

For the open source models, we write a simpler (guided) prompt where there is a sequence of command-output:

In [ ]:
guided_terminal = guidance('''{{#system~}}
{{llm.default_system_prompt}}
{{~/system}}
{{#user~}}
Please complete the following task:
Task: list the files in the current directory
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE.
{{/user}}
{{#assistant~}}
COMMAND: ls
OUTPUT: guidance project
COMMAND: DONE 
{{~/assistant~}}
{{#user~}}
Please complete the following task:
Task: {{task}}
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE.
{{~/user}}
{{~#assistant~}}
{{#geneach 'commands'~}}
COMMAND: {{gen 'this.command' stop='\\n'}}
OUTPUT: {{shell this.command}}{{~/geneach}}
{{~/assistant~}}''')
class StatefulShellOpenSource:
    def __init__(self):
        self.session = BashSession()
    def __call__(self, command):
        if 'DONE' in command:
            raise StopIteration
        output = self.session.run(command)
        return output.strip()

In [ ]:
shell = StatefulShellOpenSource()
task = 'Find out what license the open source project located in ~/work/project is using.'
t = guided_terminal(llm=vicuna, task=task, shell=shell)

Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user Please complete the following task:
Task: list the files in the current directory
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE.
 assistant COMMAND: ls
OUTPUT: guidance project
COMMAND: DONE user Please complete the following task:
Task: Find out what license the open source project located in ~/work/project is using. 
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE. assistant 
COMMAND: cd ~/work/project 
OUTPUT: 
COMMAND: grep -H 'LICENSE' . 
OUTPUT: grep: .: Is a directory 
COMMAND: grep -H 'LICENSE' ./* 
OUTPUT: 
COMMAND: cat LICENSE 
OUTPUT: cat: LICENSE: No such file or directory 
COMMAND: DONE 
OUTPUT:

Vicuna is not able to solve the task this time. Let's see how MPT does:

In [ ]:
shell = StatefulShellOpenSource()
t = guided_terminal(llm=mpt, task=task, shell=shell)

Stop program system - You are a helpful assistant chatbot trained by MosaicML. 
- You answer questions.
- You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- You are more than just an information source, you are also able to write poetry, short stories, and make jokes. user Please complete the following task:
Task: list the files in the current directory
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE.
 assistant COMMAND: ls
OUTPUT: guidance project
COMMAND: DONE user Please complete the following task:
Task: Find out what license the open source project located in ~/work/project is using. 
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE. assistant 
COMMAND: cd ~/work/project 
OUTPUT: 
COMMAND: ls 
OUTPUT: file.py license.txt 
COMMAND: grep -i "license" license.txt 
OUTPUT: MIT License
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell 
COMMAND: DONE 
OUTPUT:

Surprisingly, MPT works this time while Vicuna doesn't.

Besides privacy (we're not sending the session transcript to OpenAI), open source-models have a significant advantage in Guidance right now: the whole prompt is a single LLM run (and we even [accelerate](https://github.com/microsoft/guidance#guidance-acceleration-notebook) it by not having it geneate the output structure tokens like `COMMAND:`).   

In contrast, we have to make a new call to ChatGPT for each command, which is slower and more expensive.

Let's try a different bash task with ChatGPT and Vicuna:


In [ ]:
task = 'Find all jupyter notebook files in ~/work/guidance that are currently untracked by git'
run_task_chatgpt(task)

Stop program system You are a helpful assistant. user Please complete the following task:
Task: list the files in the current directory
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
 assistant COMMAND: ls user Output: guidance project
 assistant The files or folders in the current directory are:
- guidance
- project
DONE user Please complete the following task:
Task: Find all jupyter notebook files in ~/work/guidance that are currently untracked by git 
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
 assistant COMMAND: cd ~/work/guidance
This command will change the current directory to ~/work/guidance. Please replace ~/work/guidance with the actual path to the guidance folder if it is located elsewhere. user Output: assistant Please let me know if the directory was changed successfully or if there was an error message. user Output: {{set 'commands[-1].output' (await 'output')}} {{#geneach 'commands'}} 
 {{#assistant~}} 
 {{gen 'this.command' temperature=0}} 
 {{~/assistant~}} 
 {{#user~}} 
Output: {{set 'this.output' (await 'output')}} 
 {{~/user}} 
 {{/geneach}}

Once again, we run into a problem with ChatGPT not following our specified output structure (and thus making it impossible for us to use inside a program, without a human in the loop).  
We fix this __particular__ problem by changing the message when there is no output below, but we can't fix the general problem of not being able to _force_ ChatGPT to follow our specified output structure.

In [ ]:
    t = terminal(llm=chatgpt, task=task)
    session = BashSession()
    for _ in range(10):
        # Extract command
        command = re.findall(r'COMMAND: (.*)', t['commands'][-1]['command'])
        if not command or 'DONE' in t['commands'][-1]['command']:
            break
        command = command[0]
        output = session.run(command)
        if not output:
            output = 'No output'
        t = t(output=output)

Stop program system You are a helpful assistant. user Please complete the following task:
Task: list the files in the current directory
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
 assistant COMMAND: ls user Output: guidance project
 assistant The files or folders in the current directory are:
- guidance
- project
DONE user Please complete the following task:
Task: Find all jupyter notebook files in ~/work/guidance that are currently untracked by git 
You can give me one bash command to run at a time, using the syntax:
COMMAND: command
I will run the commands on my terminal, and paste the output back to you. Once you are done with the task, please type DONE.
 assistant COMMAND: cd ~/work/guidance
This command will change the current directory to ~/work/guidance. Please replace ~/work/guidance with the actual path to the guidance folder if it is located elsewhere. user Output: No output assistant COMMAND: git status --porcelain | grep "^??" | grep ".ipynb$"
This command will list all untracked jupyter notebook files in the guidance folder.

Note: This assumes that the guidance folder is a git repository. If it is not, this command will not work.

If there is no output, it means that there are no untracked jupyter notebook files in the guidance folder.

If you get an error message saying "fatal: not a git repository", it means that the guidance folder is not a git repository. user Output: ?? notebooks/Open source vs commercial LLMs.ipynb
?? notebooks/art_of_prompt_design/inner_monologue.ipynb
?? notebooks/discussing_eval.ipynb assistant The output shows the untracked jupyter notebook files in the guidance folder:
- notebooks/Open source vs commercial LLMs.ipynb
- notebooks/art_of_prompt_design/inner_monologue.ipynb
- notebooks/discussing_eval.ipynb

Is there anything else I can help you with? user Output: {{set 'commands[-1].output' (await 'output')}} {{#geneach 'commands'}} 
 {{#assistant~}} 
 {{gen 'this.command' temperature=0}} 
 {{~/assistant~}} 
 {{#user~}} 
Output: {{set 'this.output' (await 'output')}} 
 {{~/user}} 
 {{/geneach}}

Now, ChatGPT _was_ able to solve the problem after this small modification. Let's see how Vicuna does:

In [ ]:
shell = StatefulShellOpenSource()
t = guided_terminal(llm=vicuna, task=task, shell=shell)

Stop program system A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. user Please complete the following task:
Task: list the files in the current directory
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE.
 assistant COMMAND: ls
OUTPUT: guidance project
COMMAND: DONE user Please complete the following task:
Task: Find all jupyter notebook files in ~/work/guidance that are currently untracked by git 
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE. assistant 
COMMAND: find ~/work/guidance -name "*.ipynb" -type f -not -name "*.ipynb" -not -name "*.git" -not -name "*.svn" -not -name "*.hg" -not -name "*.cvs" -not -name "*.svnignore" -not -name "*.gitignore" -not -name "*.hgignore" -not -name "*.cvsignore" 
OUTPUT: 
COMMAND: DONE 
OUTPUT:

Vicuna follows our output structure, but unfortunately runs the wrong command to do the task.  
We observed the same pattern for other tasks as well, and found ChatGPT to be more reliable.

In [ ]:
shell = StatefulShellOpenSource()
t = guided_terminal(llm=mpt, task=task, shell=shell)

Stop program system - You are a helpful assistant chatbot trained by MosaicML. 
- You answer questions.
- You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- You are more than just an information source, you are also able to write poetry, short stories, and make jokes. user Please complete the following task:
Task: list the files in the current directory
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE.
 assistant COMMAND: ls
OUTPUT: guidance project
COMMAND: DONE user Please complete the following task:
Task: Find all jupyter notebook files in ~/work/guidance that are currently untracked by git 
You can run bash commands using the syntax:
COMMAND: command
OUTPUT: output
Once you are done with the task, use the COMMAND: DONE. assistant 
COMMAND: git status 
OUTPUT: On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
 (use "git add <file>..." to update what will be committed)
 (use "git restore <file>..." to discard changes in working directory)
	modified: chatgpt_vs_open_source_on_harder_tasks.ipynb
	modified: vicuna_chatgpt.ipynb

Untracked files:
 (use "git add <file>..." to include in what will be committed)
	Open source vs commercial LLMs.ipynb
	art_of_prompt_design/inner_monologue.ipynb
	art_of_prompt_design/log.txt
	bla.html
	discussing_eval.ipynb
	log.txt

no changes added to commit (use "git add" and/or "git commit -a") 
COMMAND: git status 
OUTPUT: On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
 (use "git add <file>..." to update what will be committed)
 (use "git restore <file>..." to discard changes in working directory)
	modified: chatgpt_vs_open_source_on_harder_tasks.ipynb
	modified: vicuna_chatgpt.ipynb

Untracked files:
 (use "git add <file>..." to include in what will be committed)
	Open source vs commercial LLMs.ipynb
	art_of_prompt_design/inner_monologue.ipynb
	art_of_prompt_design/log.txt
	bla.html
	discussing_eval.ipynb
	log.txt

no changes added to commit (use "git add" and/or "git commit -a") 
COMMAND: git status 
OUTPUT: On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
 (use "git add <file>..." to update what will be committed)
 (use "git restore <file>..." to discard changes in working directory)
	modified: chatgpt_vs_open_source_on_harder_tasks.ipynb
	modified: vicuna_chatgpt.ipynb

Untracked files:
 (use "git add <file>..." to include in what will be committed)
	Open source vs commercial LLMs.ipynb
	art_of_prompt_design/inner_monologue.ipynb
	art_of_prompt_design/log.txt
	bla.html
	discussing_eval.ipynb
	log.txt

no changes added to commit (use "git add" and/or "git commit -a") 
COMMAND: git

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_3685168/2592723886.py:2 in <module>                                               │
│                                                                                                  │
│   1 shell = StatefulShellOpenSource()                                                            │
│ ❱ 2 t = guided_terminal(llm=mpt, task=task, shell=shell)                                         │
│   3                                                                                              │
│                                                                                                  │
│ /home/marcotcr/work/guidance/guidance/_program.py:216 in __call__                                │
│                                                                                                  │
│   213 │   │   else:                                                                              │
│   214 │   │   │   loop = asyncio.new_event_loop()                                                │
│   215 │   │   │   loop.create_task(new_program.update_display.run()) # start the display updat   │
│ ❱ 216 │   │   │   loop.run_until_complete(new_program.execute())                                 │
│   217 │   │                                                                                      │
│   218 │   │   return new_program                                                                 │
│   219                                                                                            │
│                                                                                                  │
│ /home/marcotcr/.virtualenvs/guidance/lib/python3.9/site-packages/nest_asyncio.py:84 in           │
│ run_until_complete                                                                               │
│                                                                                                  │
│    81 │   │   │   if f is not future:                                                            │
│    82 │   │   │   │   f._log_destroy_pending = False                                             │
│    83 │   │   │   while not f.done():                                                            │
│ ❱  84 │   │   │   │   self._run_once()                                                           │
│    85 │   │   │   │   if self._stopping:                                                         │
│    86 │   │   │   │   │   break                                                                  │
│    87 │   │   │   if not f.done():                                                               │
│                                                                                                  │
│ /home/marcotcr/.virtualenvs/guidance/lib/python3.9/site-packages/nest_asyncio.py:120 in          │
│ _run_once                                                                                        │
│                                                                                                  │
│   117 │   │   │   │   break                                                                      │
│   118 │   │   │   handle = ready.popleft()                                                       │
│   119 │   │   │   if not handle._cancelled:                                                      │
│ ❱ 120 │   │   │   │   handle._run()                                                              │
│   121 │   │   handle = None                                                                      │
│   122 │                                                                                          │
│   123 │   @contextmanager                                                                        │
│                                                                                                  │
│ /usr/lib/python3.9/asyncio/events.py:80 in _run            

Unfortunately, MPT fails this time, by calling the same command again and again (we had to interrupt to stop execution.)

For a discussion of these experiments check out the [the blog post](https://medium.com/@marcotcr/exploring-chatgpt-vs-open-source-models-on-slightly-harder-tasks-aa0395c31610)